
<img src="https://minas.medellin.unal.edu.co/eventos/latwaves2018/images/logo_unal.png" alt="unal logo" width="500"/>

#**Trabajo Final de Maestría**
##Ingeniería de Sistemas y Computación - Perfil profundización.
Joan Gabriel Bofill Barrera

---


Inspirado en el trabajo de Tracy Porter:  https://www.kaggle.com/code/tracyporter/ell-nlp-multioutput

In [ ]:
!pip install transformers

#Import Packages

In [ ]:
import pandas as pd
import numpy as np
import os
import string
import tensorflow as tf
import matplotlib.pyplot as plt
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('brown')
from nltk.corpus import stopwords
from sklearn import preprocessing
trans = preprocessing.MinMaxScaler()

In [ ]:
from transformers import AutoModel
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import gc


In [ ]:
# Importar los datos de drive
from google.colab import drive
from google.colab import files
import io
drive.mount('/content/drive')

train = pd.read_csv('/content/drive/MyDrive/Tesis/train_modificado.csv')
test = pd.read_csv('/content/drive/MyDrive/Tesis/test_modificado.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **TfidfVectorizer**

In [ ]:
#Vectorización
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_tfIdf = vectorizer.fit_transform(train['full_text'])
X_test_tfIdf = vectorizer.transform(test['full_text'])

In [ ]:
#Targets
y_vars = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_y = train[y_vars]
test_y= test[y_vars]

In [ ]:
X_tfIdf.shape, X_test_tfIdf.shape, train_y.shape

((3128, 21866), (783, 21866), (3128, 6))

In [ ]:
%%time
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

chain = MultiOutputRegressor(SVR())
chain.fit(X_tfIdf, train_y)
print(chain.score(X_tfIdf,train_y))

0.9006035027808883
CPU times: user 2min 33s, sys: 359 ms, total: 2min 33s
Wall time: 2min 33s


In [ ]:
predictions = chain.predict(X_test_tfIdf)
predictions

array([[3.08284389, 2.8042394 , 2.86619329, 2.77057351, 2.7016355 ,
        2.75300148],
       [2.83100049, 3.10843497, 3.06087221, 2.8489132 , 3.1898067 ,
        3.0373576 ],
       [3.62789113, 3.39614584, 3.56019653, 3.47768361, 3.15908459,
        3.48515429],
       ...,
       [2.74908861, 2.88292261, 2.74355538, 2.71661696, 2.9876424 ,
        2.7797491 ],
       [3.77018806, 3.68147778, 4.02222417, 4.02239427, 3.60395594,
        3.74546067],
       [2.97977179, 2.54240195, 2.9614465 , 2.56740004, 2.30522785,
        2.72081664]])

In [ ]:
preds = pd.DataFrame(predictions, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= test_y.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,3.082844,2.804239,2.866193,2.770574,2.701635,2.753001
1,3.0,2.0,3.0,3.5,3.0,3.0,2.831000,3.108435,3.060872,2.848913,3.189807,3.037358
2,4.0,4.0,3.0,4.0,4.0,4.0,3.627891,3.396146,3.560197,3.477684,3.159085,3.485154
3,3.0,3.0,3.5,3.0,3.5,3.5,2.806860,3.104666,3.211448,2.980827,2.909218,2.837434
4,3.5,3.5,3.5,3.5,3.0,3.5,3.102654,3.018690,3.156848,2.963631,3.094020,3.084879
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.425409,3.366550,3.536314,3.423858,3.542861,3.128241
779,2.5,2.5,3.0,3.0,2.5,2.5,3.126218,3.146656,3.036983,3.126466,3.144774,3.206365
780,2.0,3.0,3.0,3.0,3.0,2.5,2.749089,2.882923,2.743555,2.716617,2.987642,2.779749
781,4.0,3.5,4.0,3.5,3.5,4.0,3.770188,3.681478,4.022224,4.022394,3.603956,3.745461


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,3.082844,2.804239,2.866193,2.770574,2.701635,2.753001,0.006863,0.092562,0.134098,0.593784,0.492292,0.567011
1,3.0,2.0,3.0,3.5,3.0,3.0,2.831000,3.108435,3.060872,2.848913,3.189807,3.037358,0.028561,1.228628,0.003705,0.423914,0.036027,0.001396
2,4.0,4.0,3.0,4.0,4.0,4.0,3.627891,3.396146,3.560197,3.477684,3.159085,3.485154,0.138465,0.364640,0.313820,0.272814,0.707139,0.265066
3,3.0,3.0,3.5,3.0,3.5,3.5,2.806860,3.104666,3.211448,2.980827,2.909218,2.837434,0.037303,0.010955,0.083262,0.000368,0.349023,0.438994
4,3.5,3.5,3.5,3.5,3.0,3.5,3.102654,3.018690,3.156848,2.963631,3.094020,3.084879,0.157884,0.231659,0.117753,0.287691,0.008840,0.172325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.425409,3.366550,3.536314,3.423858,3.542861,3.128241,2.031791,0.750908,0.287632,0.179656,0.001837,1.272928
779,2.5,2.5,3.0,3.0,2.5,2.5,3.126218,3.146656,3.036983,3.126466,3.144774,3.206365,0.392149,0.418164,0.001368,0.015994,0.415734,0.498952
780,2.0,3.0,3.0,3.0,3.0,2.5,2.749089,2.882923,2.743555,2.716617,2.987642,2.779749,0.561134,0.013707,0.065764,0.080306,0.000153,0.078260
781,4.0,3.5,4.0,3.5,3.5,4.0,3.770188,3.681478,4.022224,4.022394,3.603956,3.745461,0.052814,0.032934,0.000494,0.272896,0.010807,0.064790


In [ ]:
mse = results.mean();mse

se_cohesion       0.307468
se_syntax         0.298103
se_vocabulary     0.240989
se_phraseology    0.284466
se_grammar        0.347870
se_conventions    0.322797
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.554498
se_syntax         0.545988
se_vocabulary     0.490906
se_phraseology    0.533354
se_grammar        0.589805
se_conventions    0.568153
dtype: float64

In [ ]:
rmse.mean()

0.5471171892115767

#**Word2Vec**

In [ ]:
from nltk.tokenize import word_tokenize
# Tokeniza los ensayos quitanto las stopwords
train['tokens'] = train['full_text'].apply(word_tokenize)
test['tokens'] = test['full_text'].apply(word_tokenize)

In [ ]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from gensim.test.utils import common_texts

model_w2v = Word2Vec(sentences=train["tokens"], vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
tokenized_texts = train["tokens"]
tokenized_texts_test = test["tokens"]

# Function to calculate document embeddings
def document_embedding(tokens, model):
    # Filter out words that are not in the model's vocabulary
    words_in_vocab = [word for word in tokens if word in model.wv.key_to_index]

    # If no words are in the model's vocabulary, return a zero vector
    if not words_in_vocab:
        return [0.0] * model.vector_size

    # Calculate the mean of word vectors for words in the model's vocabulary
    doc_vector = sum(model.wv[word] for word in words_in_vocab) / len(words_in_vocab)

    return doc_vector

In [ ]:
# Calculate document embeddings for each text
document_embeddings = [document_embedding(tokens, model_w2v) for tokens in tokenized_texts]
document_embeddings_test = [document_embedding(tokens, model_w2v) for tokens in tokenized_texts_test]

In [ ]:
%%time
chain = MultiOutputRegressor(SVR())
chain.fit(document_embeddings, train_y)
print(chain.score(document_embeddings,train_y))

0.3183081626548216
CPU times: user 10.3 s, sys: 46.9 ms, total: 10.4 s
Wall time: 10.3 s


In [ ]:
predictions = chain.predict(document_embeddings_test)
predictions

array([[2.9240866 , 2.74072376, 2.78479516, 2.74474117, 2.41077162,
        2.84095071],
       [2.92519805, 2.96967329, 2.97108242, 2.9493681 , 2.83670722,
        2.94324246],
       [3.26552047, 3.21727682, 3.45917478, 3.32569284, 3.00971756,
        3.26799353],
       ...,
       [2.52584753, 2.59108355, 2.81493661, 2.67552473, 2.83822516,
        2.77307819],
       [3.61985027, 3.46670976, 3.74837787, 3.77995996, 3.45833395,
        3.47098675],
       [2.85810086, 2.49312525, 2.7881233 , 2.56835128, 2.17286007,
        2.70866323]])

In [ ]:
preds = pd.DataFrame(predictions, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= test_y.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.924087,2.740724,2.784795,2.744741,2.410772,2.840951
1,3.0,2.0,3.0,3.5,3.0,3.0,2.925198,2.969673,2.971082,2.949368,2.836707,2.943242
2,4.0,4.0,3.0,4.0,4.0,4.0,3.265520,3.217277,3.459175,3.325693,3.009718,3.267994
3,3.0,3.0,3.5,3.0,3.5,3.5,3.042254,2.890706,3.176477,3.080232,2.911667,3.114430
4,3.5,3.5,3.5,3.5,3.0,3.5,3.141429,3.124358,3.223954,3.202694,3.263747,3.211003
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.482915,3.388624,3.618323,3.525318,3.482126,3.402217
779,2.5,2.5,3.0,3.0,2.5,2.5,3.341677,2.966797,3.112824,3.084141,3.288309,3.040855
780,2.0,3.0,3.0,3.0,3.0,2.5,2.525848,2.591084,2.814937,2.675525,2.838225,2.773078
781,4.0,3.5,4.0,3.5,3.5,4.0,3.619850,3.466710,3.748378,3.779960,3.458334,3.470987


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.924087,2.740724,2.784795,2.744741,2.410772,2.840951,0.005763,0.057948,0.081108,0.554639,0.168733,0.707198
1,3.0,2.0,3.0,3.5,3.0,3.0,2.925198,2.969673,2.971082,2.949368,2.836707,2.943242,0.005595,0.940266,0.000836,0.303195,0.026665,0.003221
2,4.0,4.0,3.0,4.0,4.0,4.0,3.265520,3.217277,3.459175,3.325693,3.009718,3.267994,0.539460,0.612656,0.210841,0.454690,0.980659,0.535833
3,3.0,3.0,3.5,3.0,3.5,3.5,3.042254,2.890706,3.176477,3.080232,2.911667,3.114430,0.001785,0.011945,0.104667,0.006437,0.346136,0.148665
4,3.5,3.5,3.5,3.5,3.0,3.5,3.141429,3.124358,3.223954,3.202694,3.263747,3.211003,0.128573,0.141107,0.076201,0.088391,0.069563,0.083519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.482915,3.388624,3.618323,3.525318,3.482126,3.402217,2.199038,0.789652,0.382323,0.275959,0.000319,1.966213
779,2.5,2.5,3.0,3.0,2.5,2.5,3.341677,2.966797,3.112824,3.084141,3.288309,3.040855,0.708420,0.217900,0.012729,0.007080,0.621431,0.292524
780,2.0,3.0,3.0,3.0,3.0,2.5,2.525848,2.591084,2.814937,2.675525,2.838225,2.773078,0.276516,0.167213,0.034248,0.105284,0.026171,0.074572
781,4.0,3.5,4.0,3.5,3.5,4.0,3.619850,3.466710,3.748378,3.779960,3.458334,3.470987,0.144514,0.001108,0.063314,0.078378,0.001736,0.279855


In [ ]:
mse = results.mean();mse

se_cohesion       0.341023
se_syntax         0.331952
se_vocabulary     0.258220
se_phraseology    0.318659
se_grammar        0.366954
se_conventions    0.360678
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.583972
se_syntax         0.576152
se_vocabulary     0.508154
se_phraseology    0.564499
se_grammar        0.605767
se_conventions    0.600565
dtype: float64

In [ ]:
rmse.mean()

0.5731847424187779

#**Glove**

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [ ]:
import gensim.downloader as api
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
glove_model = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
def essay_to_vector(essay, model):
    return np.mean([model[word] for word in essay if word in model], axis=0)

In [ ]:
essay_vectors = [essay_to_vector(essay, glove_model) for essay in tokenized_texts]
essay_vectors_test = [essay_to_vector(essay, glove_model) for essay in tokenized_texts_test]

In [ ]:
essay_vectors

In [ ]:
%%time
chain = MultiOutputRegressor(SVR())
chain.fit(essay_vectors, train_y)
print(chain.score(essay_vectors,train_y))

0.17186470765382908
CPU times: user 10.3 s, sys: 107 ms, total: 10.4 s
Wall time: 10.4 s


In [ ]:
predictions = chain.predict(essay_vectors_test)
predictions

array([[2.93582806, 2.79396638, 2.9377243 , 2.83173119, 2.56046174,
        2.85467265],
       [2.93753355, 2.89769747, 3.06528341, 3.02749666, 2.89812466,
        3.00181464],
       [3.17561667, 3.0662673 , 3.22791475, 3.18824908, 2.99910221,
        3.17939224],
       ...,
       [2.86614193, 2.92541851, 3.13112558, 3.09128869, 3.01391895,
        2.81938798],
       [3.28804992, 3.20740511, 3.37518262, 3.35428219, 3.17264734,
        3.27341172],
       [3.10126605, 2.87104575, 3.23599324, 2.97904459, 2.78821188,
        2.95906878]])

In [ ]:
preds = pd.DataFrame(predictions, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= test_y.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,2.935828,2.793966,2.937724,2.831731,2.560462,2.854673
1,3.0,2.0,3.0,3.5,3.0,3.0,2.937534,2.897697,3.065283,3.027497,2.898125,3.001815
2,4.0,4.0,3.0,4.0,4.0,4.0,3.175617,3.066267,3.227915,3.188249,2.999102,3.179392
3,3.0,3.0,3.5,3.0,3.5,3.5,3.026258,2.937006,3.171962,3.064587,2.900274,2.991951
4,3.5,3.5,3.5,3.5,3.0,3.5,3.153862,3.029657,3.220103,3.169226,3.100884,3.094637
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.288601,3.212390,3.378943,3.329092,3.279431,3.185176
779,2.5,2.5,3.0,3.0,2.5,2.5,3.237694,3.131471,3.229576,3.254535,3.246004,3.293280
780,2.0,3.0,3.0,3.0,3.0,2.5,2.866142,2.925419,3.131126,3.091289,3.013919,2.819388
781,4.0,3.5,4.0,3.5,3.5,4.0,3.288050,3.207405,3.375183,3.354282,3.172647,3.273412


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,2.935828,2.793966,2.937724,2.831731,2.560462,2.854673,0.004118,0.086416,0.191603,0.691777,0.314117,0.730465
1,3.0,2.0,3.0,3.5,3.0,3.0,2.937534,2.897697,3.065283,3.027497,2.898125,3.001815,0.003902,0.805861,0.004262,0.223259,0.010379,0.000003
2,4.0,4.0,3.0,4.0,4.0,4.0,3.175617,3.066267,3.227915,3.188249,2.999102,3.179392,0.679608,0.871857,0.051945,0.658940,1.001796,0.673397
3,3.0,3.0,3.5,3.0,3.5,3.5,3.026258,2.937006,3.171962,3.064587,2.900274,2.991951,0.000690,0.003968,0.107609,0.004171,0.359671,0.258113
4,3.5,3.5,3.5,3.5,3.0,3.5,3.153862,3.029657,3.220103,3.169226,3.100884,3.094637,0.119812,0.221222,0.078342,0.109412,0.010178,0.164319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,3.288601,3.212390,3.378943,3.329092,3.279431,3.185176,1.660492,0.507500,0.143598,0.108301,0.048651,1.404642
779,2.5,2.5,3.0,3.0,2.5,2.5,3.237694,3.131471,3.229576,3.254535,3.246004,3.293280,0.544192,0.398756,0.052705,0.064788,0.556523,0.629293
780,2.0,3.0,3.0,3.0,3.0,2.5,2.866142,2.925419,3.131126,3.091289,3.013919,2.819388,0.750202,0.005562,0.017194,0.008334,0.000194,0.102009
781,4.0,3.5,4.0,3.5,3.5,4.0,3.288050,3.207405,3.375183,3.354282,3.172647,3.273412,0.506873,0.085612,0.390397,0.021234,0.107160,0.527931


In [ ]:
mse = results.mean();mse

se_cohesion       0.366049
se_syntax         0.360191
se_vocabulary     0.281386
se_phraseology    0.365823
se_grammar        0.422483
se_conventions    0.386525
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.605020
se_syntax         0.600159
se_vocabulary     0.530458
se_phraseology    0.604833
se_grammar        0.649987
se_conventions    0.621711
dtype: float64

In [ ]:
rmse.mean()

0.6020280544905652

**Normalizando las Características extraídas**

In [ ]:
#Toma las características extraídas
variables = [var for var in train.columns if var not in ['full_text', 'text_id', 'cohesion','syntax','vocabulary','phraseology','grammar','conventions','tokens']]

In [ ]:
variables

['Unnamed: 0',
 'word_count',
 'diff',
 'n_capital',
 'n_punct',
 'spelling_errors',
 'n_unique',
 'n_unique_n_stop',
 'n_n_word',
 'noun_phrase_count',
 'PRON',
 'VERB',
 'SCONJ',
 'NOUN',
 'AUX',
 'ADP',
 'PUNCT',
 'PART',
 'CCONJ',
 'ADV',
 'DET',
 'ADJ',
 'SPACE',
 'PROPN',
 'NUM',
 'INTJ',
 'SYM',
 'X',
 'polarity',
 'subjectivity',
 'spell_score',
 'av_sent_len',
 'max_sent_len',
 'min_sent_len',
 'med_sent_len',
 'std_sent_len',
 'num_sent',
 'compound',
 'negative',
 'positive',
 'neutral',
 'char_len']

In [ ]:
#Normaliza las variables para el conjunto de entrenamiento
X_train_custom=trans.fit_transform(train[variables].values)
X_train_custom.shape

(3128, 42)

In [ ]:
#Normaliza las variables para el conjunto de prueba
X_test_custom=trans.fit_transform(test[variables].values)
X_test_custom.shape

(783, 42)

In [ ]:
%%time
chain = MultiOutputRegressor(SVR())
chain.fit(X_train_custom, train_y)
print(chain.score(X_train_custom,train_y))

0.35512656460861014
CPU times: user 7.78 s, sys: 106 ms, total: 7.89 s
Wall time: 7.86 s


In [ ]:
predictions = chain.predict(X_test_custom)
predictions

array([[1.72046038, 1.46014261, 1.97901403, 1.63027287, 1.14930273,
        0.82580102],
       [2.53747013, 2.20966964, 2.68796317, 2.71577737, 2.54173861,
        2.43833236],
       [3.5256233 , 3.2786692 , 3.44345035, 3.39567661, 3.1850326 ,
        3.3956367 ],
       ...,
       [2.16458032, 2.13965009, 2.43667459, 2.14714835, 1.85082655,
        2.50905601],
       [3.74096993, 3.61640171, 3.771894  , 3.82635504, 3.65381168,
        3.62023824],
       [3.12069279, 2.75501735, 3.11332985, 2.84602997, 2.72046294,
        2.86732642]])

In [ ]:
preds = pd.DataFrame(predictions, columns = ["cohesion_pred",	"syntax_pred",	"vocabulary_pred",	"phraseology_pred",	"grammar_pred",	"conventions_pred"])

In [ ]:
y_test= test_y.reset_index(drop=True)

In [ ]:
preresults =pd.concat([y_test, preds], axis=1);preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred
0,3.0,2.5,2.5,2.0,2.0,2.0,1.720460,1.460143,1.979014,1.630273,1.149303,0.825801
1,3.0,2.0,3.0,3.5,3.0,3.0,2.537470,2.209670,2.687963,2.715777,2.541739,2.438332
2,4.0,4.0,3.0,4.0,4.0,4.0,3.525623,3.278669,3.443450,3.395677,3.185033,3.395637
3,3.0,3.0,3.5,3.0,3.5,3.5,2.743940,2.783565,2.851202,2.828607,2.710063,2.684661
4,3.5,3.5,3.5,3.5,3.0,3.5,2.583973,2.746232,2.590883,2.539264,2.376575,2.310357
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,2.701550,2.446683,2.810258,2.742743,2.559568,1.788951
779,2.5,2.5,3.0,3.0,2.5,2.5,2.707524,2.667489,2.702799,2.681092,2.589584,2.627642
780,2.0,3.0,3.0,3.0,3.0,2.5,2.164580,2.139650,2.436675,2.147148,1.850827,2.509056
781,4.0,3.5,4.0,3.5,3.5,4.0,3.740970,3.616402,3.771894,3.826355,3.653812,3.620238


In [ ]:
preresults["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
preresults["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
preresults["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
preresults["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
preresults["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
preresults["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
results = pd.DataFrame()
results["se_cohesion"]=(preresults["cohesion"]-preresults["cohesion_pred"])**2
results["se_syntax"]=(preresults["syntax"]-preresults["syntax_pred"])**2
results["se_vocabulary"]=(preresults["vocabulary"]-preresults["vocabulary_pred"])**2
results["se_phraseology"]=(preresults["phraseology"]-preresults["phraseology_pred"])**2
results["se_grammar"]=(preresults["grammar"]-preresults["grammar_pred"])**2
results["se_conventions"]=(preresults["conventions"]-preresults["conventions_pred"])**2

In [ ]:
preresults

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cohesion_pred,syntax_pred,vocabulary_pred,phraseology_pred,grammar_pred,conventions_pred,se_cohesion,se_syntax,se_vocabulary,se_phraseology,se_grammar,se_conventions
0,3.0,2.5,2.5,2.0,2.0,2.0,1.720460,1.460143,1.979014,1.630273,1.149303,0.825801,1.637222,1.081303,0.271426,0.136698,0.723686,1.378743
1,3.0,2.0,3.0,3.5,3.0,3.0,2.537470,2.209670,2.687963,2.715777,2.541739,2.438332,0.213934,0.043961,0.097367,0.615005,0.210004,0.315471
2,4.0,4.0,3.0,4.0,4.0,4.0,3.525623,3.278669,3.443450,3.395677,3.185033,3.395637,0.225033,0.520318,0.196648,0.365207,0.664172,0.365255
3,3.0,3.0,3.5,3.0,3.5,3.5,2.743940,2.783565,2.851202,2.828607,2.710063,2.684661,0.065567,0.046844,0.420939,0.029376,0.624001,0.664778
4,3.5,3.5,3.5,3.5,3.0,3.5,2.583973,2.746232,2.590883,2.539264,2.376575,2.310357,0.839105,0.568166,0.826494,0.923014,0.388659,1.415250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2.0,2.5,3.0,3.0,3.5,2.0,2.701550,2.446683,2.810258,2.742743,2.559568,1.788951,0.492173,0.002843,0.036002,0.066181,0.884412,0.044542
779,2.5,2.5,3.0,3.0,2.5,2.5,2.707524,2.667489,2.702799,2.681092,2.589584,2.627642,0.043066,0.028053,0.088329,0.101703,0.008025,0.016293
780,2.0,3.0,3.0,3.0,3.0,2.5,2.164580,2.139650,2.436675,2.147148,1.850827,2.509056,0.027087,0.740202,0.317336,0.727356,1.320600,0.000082
781,4.0,3.5,4.0,3.5,3.5,4.0,3.740970,3.616402,3.771894,3.826355,3.653812,3.620238,0.067097,0.013549,0.052032,0.106508,0.023658,0.144219


In [ ]:
mse = results.mean();mse

se_cohesion       0.428877
se_syntax         0.415720
se_vocabulary     0.310565
se_phraseology    0.416657
se_grammar        0.547753
se_conventions    0.565362
dtype: float64

In [ ]:
rmse = np.sqrt(mse);rmse

se_cohesion       0.654887
se_syntax         0.644763
se_vocabulary     0.557283
se_phraseology    0.645490
se_grammar        0.740103
se_conventions    0.751906
dtype: float64

In [ ]:
rmse.mean()

0.6657386748892503